In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime
from time import sleep

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats,lngs)
lats_lngs

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lats_lngs)

In [4]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=92e5b44c66f318665ac591f4d5b9ab42


In [5]:
# Create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

# If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

755

In [6]:
# Create an empty list to hold the weather data
city_data = []
# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count+=1
        record_count+=1
    # Create endpoint URL with each city
    sleep(2)
    city_url = url + "&q=" + city.replace(" ", "+")
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count+=1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        weather_desc = city_weather["weather"][0]["description"]
                
        # Append the city information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Description": weather_desc})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping....")
        pass

# Indicate that Data Loading is complete
print("----------------------------------")
print("Data Retrieval Complete           ")
print("----------------------------------")


cessing Record 323 of Set 7 | palm coast
Processing Record 324 of Set 7 | kaitangata
Processing Record 325 of Set 7 | potosi
Processing Record 326 of Set 7 | thunder bay
Processing Record 327 of Set 7 | punta alta
Processing Record 328 of Set 7 | vila franca do campo
Processing Record 329 of Set 7 | constitucion
Processing Record 330 of Set 7 | nyurba
Processing Record 331 of Set 7 | ilulissat
Processing Record 332 of Set 7 | hasaki
Processing Record 333 of Set 7 | ginda
Processing Record 334 of Set 7 | saldanha
Processing Record 335 of Set 7 | touros
Processing Record 336 of Set 7 | shelton
Processing Record 337 of Set 7 | tanete
Processing Record 338 of Set 7 | san julian
Processing Record 339 of Set 7 | grants
Processing Record 340 of Set 7 | husavik
Processing Record 341 of Set 7 | makakilo city
Processing Record 342 of Set 7 | high level
Processing Record 343 of Set 7 | amderma
City not found. Skipping....
Processing Record 344 of Set 7 | araouane
Processing Record 345 of Set 7 | 

In [35]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
# Rearranding the column sequence
new_column_order = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Description"]
city_data_df = city_data_df[new_column_order]
# Renaming Description column to Current Description
city_data_df.columns = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hobart,AU,-42.8794,147.3294,68.00,63,75,17.27,broken clouds
1,Mataura,NZ,-46.1927,168.8643,81.86,44,5,1.34,clear sky
2,Busselton,AU,-33.6500,115.3333,62.01,95,100,3.00,light rain
3,Ancud,CL,-41.8697,-73.8203,62.60,45,0,10.36,clear sky
4,Fuerte Olimpo,PY,-21.0415,-57.8738,70.05,64,0,4.12,clear sky
5,Hermanus,ZA,-34.4187,19.2345,66.99,94,98,4.76,overcast clouds
6,Verkhoyansk,RU,67.5447,133.3850,-22.41,83,98,1.77,overcast clouds
7,Port Alfred,ZA,-33.5906,26.8910,66.99,87,33,4.00,scattered clouds
8,Castro,BR,-24.7911,-50.0119,61.25,92,42,10.87,scattered clouds
9,Kodiak,US,57.7900,-152.4072,35.60,64,1,5.75,clear sky


In [38]:
# Create output file CSV
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")